### Design for Co-Factor Swapping

The objective of this analysis is to investigate if productivity can be improved by swapping NADPH for NADH as a cofactor in the model, in order to optimize flux through reactions requiring CYP71AV1. The co-factor swap analysis will determine if an increase in the available NADPH will lead to increased production of dihydroartemisinic acid, our target compound. The hypothesis proposed, that changing co-factor specificity may increase production of a target compound, is supported by work conducted by King and Feist (2014) who showed that co-factor swapping in central carbon metabolism has the potential to improve (theoretical) yields of some amino acids in _S. cerevisiae_. The main function used in this analysis is CofactorSwapAnalysis. In order to enhance the availability of a cofactor that is rate-limiting for the generation of a target metabolite and raise its theoretical yield, CofactorSwapOptimization uses an algorithm to determine which reactions in the model can undergo a swap in cofactor specificity.

In [1]:
# Initialising all required packages
from cobra.io import read_sbml_model, write_sbml_model #importing functions needed
from cobra import Reaction, Metabolite
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

In [2]:
model = read_sbml_model("data/iYO844_modified.xml")
model

Name,iYO844
Memory address,0x01eabb20c708
Number of metabolites,996
Number of reactions,1258
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


In [3]:
with model:
    base_max_growth = model.slim_optimize()
    print(base_max_growth)
    model.objective = model.reactions.ALDH1_CYP71AV1
    base_max_production = model.slim_optimize()
    print(base_max_production)

0.11796638932239784
0.36342857142856866


In [4]:
model.solver = "glpk" #specifying the solver for optimizing to be used as GLPK (GNU Linear Programming Kit)

biomass = model.reactions.BIOMASS_BS_10 #defining the biomass from the reaction describing biomass formation in our model

biomass.lower_bound = 0.1 # WHY ARE WE SETTING THE LOWER BOUND FOR THIS

demand = model.reactions.DM_dihydroartaci_c # Defining the demand for our target reaction of dihydroartemisinic acid
#DM_ defines the demand for us.

model.objective = demand # Setting the objective of the model to the variable demand

In [5]:
#Her skal vi have vores nye boundary, som optimerer for produktionen af vores ønskede stof
# prod_yield = product_yield(demand, model.reactions.EX_glc__D_e) - this has been done by other groups, but for us, yields only the biomass as result
# prod_yield = product_yield(model.reactions.ALDH1_CYP71AV1, biomass) - gives same result as below
prod_yield = product_yield(model.reactions.DM_dihydroartaci_c, biomass) 
prod_yield

In [6]:
# Swapping NADPH to NADH                                                                               
#swap_opt = CofactorSwapOptimization(model=model, objective_function = prod_yield, plot=True, cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']))
swap_opt = CofactorSwapOptimization(model=model, objective_function = prod_yield, plot=True)

In [7]:
%%time
result = swap_opt.run(max_size=5) #can be changed, but I'M NOT QUITE SURE WHAT EXACTLY CHANGES (looks like time in min)
#we get no result right now, meaning nothing was found.... Can that be right?

Starting optimization at Tue, 21 Nov 2023 15:22:35


HBox()

Finished after 00:00:26
Wall time: 49 s


In [8]:
result

,index,targets,fitness


The co-factor swap analysis reveals no reactions that, upon changing co-factor specificity, would lead to an increased theoretical yield of dihydroartesiminic acid.

We will now investigate if swapping the co-factor specificity of CYP71AV1 from NADP+NADH to NAD+NADH will allow an increased theoretical yield of dihydroartesiminic acid. Several of the introduced reactions utilize CYP71AV1; reaction 3, 4 and 6.

In [9]:
iYO844_modified_swap = model.copy() # Generating a copy of iY0844_modified in a variable to add the reactions to the new model

In [10]:
# Creating the reaction
reaction_3_swap = Reaction('CYP71AV1_swap') # CYP71AV1: amorphadiene oxidase

# Adding the metabolites based on the stoichiometric reaction
reaction_3_swap.add_metabolites({model.metabolites.get_by_id('amorph_c'): -1,
                            model.metabolites.get_by_id('o2_c'): -1,
                            model.metabolites.get_by_id('nad_c'): -1,
                            model.metabolites.get_by_id('artalc_c'): 1, 
                            model.metabolites.get_by_id('nadh_c'): 1,
                            model.metabolites.get_by_id('h2o_c'): 1})

print(reaction_3_swap.build_reaction_string()) # Printing how the reaction looks
iYO844_modified_swap.add_reactions([reaction_3_swap]) # Adding the reaction to the new iY0844 model

amorph_c + nad_c + o2_c --> artalc_c + h2o_c + nadh_c


In [11]:
# Creating the reaction
reaction_6_swap = Reaction('ALDH1_CYP71AV1_swap') # ALDH1: aldehyde dehydrogenase 1, CYP71AV1: amorphadiene oxidase

# Adding the metabolites based on the stoichiometric reaction
reaction_6_swap.add_metabolites({model.metabolites.get_by_id('artalc_c'): -1,
                            model.metabolites.get_by_id('o2_c'): -1,
                            model.metabolites.get_by_id('nad_c'): -1,
                            model.metabolites.get_by_id('dihydroartaci_c'): 1, 
                            model.metabolites.get_by_id('nadh_c'): 1,
                            model.metabolites.get_by_id('h2o_c'): 1})

print(reaction_6_swap.build_reaction_string()) # Printing how the reaction looks
iYO844_modified_swap.add_reactions([reaction_6_swap]) # Adding the reaction to the new iY0844 model

artalc_c + nad_c + o2_c --> dihydroartaci_c + h2o_c + nadh_c


In [12]:
model.reactions.CYP71AV1.knock_out()
#model.reactions.CYP71AV1

In [13]:
model.reactions.ADH1_CYP71AV1.knock_out()
#model.reactions.ADH1_CYP71AV1

In [14]:
model.reactions.ALDH1_CYP71AV1.knock_out()
#model.reactions.ALDH1_CYP71AV1

In [18]:
with iYO844_modified_swap:
    iYO844_modified_swap.objective = iYO844_modified_swap.reactions.BIOMASS_BS_10
    swap_max_growth = iYO844_modified_swap.slim_optimize()
    print(swap_max_growth)
    iYO844_modified_swap.objective = iYO844_modified_swap.reactions.ALDH1_CYP71AV1_swap
    swap_max_production = iYO844_modified_swap.slim_optimize()
    print(swap_max_production)

0.11796638932239606
0.05535050485715387


In [19]:
print("Max growth has changed by", round((swap_max_growth-(base_max_growth))/(base_max_growth) * 100, 2), "percent")
print("Max production has changed by", round((swap_max_production-(base_max_production))/(base_max_production) * 100, 2), "percent")

Max growth has changed by -0.0 percent
Max production has changed by -84.77 percent


### Saving the model
The current model is saved as "iYO844_modified_swap.xml" in the data folder.

In [ ]:
write_sbml_model(iYO844_modified_swap, 'data/iYO844_modified_swap.xml')